In [26]:
import pandas as pd
import os

## Load raw data from the RAW folder

In [27]:
raw_data_file = '../RAW/data/users.csv'  # Path to your CSV file in the RAW folder
users_raw = pd.read_csv(raw_data_file)
departments_raw_file = '../RAW/data/departments.csv'  # Path to your CSV file for departments
departments_raw = pd.read_csv(departments_raw_file)
teams_raw_file = '../RAW/data/teams.csv'  # Path to your CSV file for teams
# Load the raw team data
teams_raw = pd.read_csv(teams_raw_file)
# load the course data
courses_raw_file = '../RAW/data/courses.csv'  # Path to your CSV file for courses
courses_raw = pd.read_csv(courses_raw_file)
learning_materials_raw_file = '../RAW/data/learning_materials.csv'  # Path to your CSV file for learning materials
# Load the raw learning materials data
learning_materials_raw = pd.read_csv(learning_materials_raw_file)


# Load raw data from the RAW folder
user_progress_raw_file = '../RAW/data/user_progress.csv'  # Path to your CSV file for user progress
quizzes_raw_file = '../RAW/data/quizzes.csv'  # Path to your CSV file for quizzes
questions_raw_file = '../RAW/data/questions.csv'  # Path to your CSV file for questions
results_raw_file = '../RAW/data/results.csv'  # Path to your CSV file for results
posts_raw_file = '../RAW/data/posts.csv'  # Path to your CSV file for posts
feedbacks_raw_file = '../RAW/data/feedbacks.csv'  # Path to your CSV file for feedback
user_sessions_raw_file = '../RAW/data/user_sessions.csv'  # Path to your CSV file for user sessions

In [28]:
# Create PREP folder if it doesn't exist
os.makedirs('./data', exist_ok=True)

## Cleaning: Remove duplicates and NaN values for users and load data

In [29]:

users_cleaned = users_raw.drop_duplicates(subset=['email'])
users_cleaned = users_cleaned.dropna(subset=['name', 'email'])

# some transformation
users_cleaned['name'] = users_cleaned['name'].str.title()  # Capitalize names
users_cleaned['email'] = users_cleaned['email'].str.lower()  # Lowercase 


# loading to prep layer
prep_file_path = './data/prep_users.csv'
users_cleaned.to_csv(prep_file_path, index=False)

## For department and team

In [30]:
departments_cleaned = departments_raw.drop_duplicates(subset=['name'])
departments_cleaned = departments_cleaned.dropna(subset=['name'])


prep_departments_file_path = './data/prep_departments.csv'
departments_cleaned.to_csv(prep_departments_file_path, index=False)

# Cleaning for Teams
teams_cleaned = teams_raw.drop_duplicates(subset=['name'])
teams_cleaned = teams_cleaned.dropna(subset=['name', 'departmentId'])

# Optional: Check if departmentId exists in the departments dataset
# This step is to ensure referential integrity (if needed)
if 'departmentId' in teams_cleaned.columns:
    valid_departments = departments_cleaned['id'].unique()
    teams_cleaned = teams_cleaned[teams_cleaned['departmentId'].isin(valid_departments)]

# Saving cleaned teams data
prep_teams_file_path = './data/prep_teams.csv'
teams_cleaned.to_csv(prep_teams_file_path, index=False)

## For course and learning material

In [31]:
courses_cleaned = courses_raw.drop_duplicates(subset=['name'])
courses_cleaned = courses_cleaned.dropna(subset=['name', 'content', 'teamId'])

# Check if teamId exists in the teams dataset
teams_file_path = './data/prep_teams.csv'
if 'teamId' in courses_cleaned.columns:
    teams_df = pd.read_csv(teams_file_path)
    valid_teams = teams_df['id'].unique()
    courses_cleaned = courses_cleaned[courses_cleaned['teamId'].isin(valid_teams)]

# Saving cleaned course data
prep_courses_file_path = './data/prep_courses.csv'
courses_cleaned.to_csv(prep_courses_file_path, index=False)


# Cleaning for Learning Materials
learning_materials_cleaned = learning_materials_raw.drop_duplicates(subset=['title'])
learning_materials_cleaned = learning_materials_cleaned.dropna(subset=['title', 'type', 'content', 'courseId'])

# Check if courseId exists in the courses dataset
if 'courseId' in learning_materials_cleaned.columns:
    valid_courses = courses_cleaned['id'].unique()
    learning_materials_cleaned = learning_materials_cleaned[learning_materials_cleaned['courseId'].isin(valid_courses)]

# Saving cleaned learning materials data
prep_learning_materials_file_path = './data/prep_learning_materials.csv'
learning_materials_cleaned.to_csv(prep_learning_materials_file_path, index=False)


## For user progress

In [32]:
user_progress_raw = pd.read_csv(user_progress_raw_file)

# Cleaning UserProgress
user_progress_cleaned = user_progress_raw.drop_duplicates(subset=['userId', 'learningMaterialId'])
user_progress_cleaned = user_progress_cleaned.dropna(subset=['userId', 'learningMaterialId'])

# Convert completedAt to datetime
user_progress_cleaned['completedAt'] = pd.to_datetime(user_progress_cleaned['completedAt'], errors='coerce')

# Saving cleaned user progress data
prep_user_progress_file_path = './data/prep_user_progress.csv'
user_progress_cleaned.to_csv(prep_user_progress_file_path, index=False)
print(f"User Progress data cleaned and saved to {prep_user_progress_file_path} successfully.")

User Progress data cleaned and saved to ./data/prep_user_progress.csv successfully.


## For quiz data

In [33]:
quizzes_raw = pd.read_csv(quizzes_raw_file)

# Cleaning Quizzes
quizzes_cleaned = quizzes_raw.drop_duplicates(subset=['courseId'])
quizzes_cleaned = quizzes_cleaned.dropna(subset=['courseId'])

# Check if courseId exists in the courses dataset
courses_df = pd.read_csv('./data/prep_courses.csv')
valid_courses = courses_df['id'].unique()
quizzes_cleaned = quizzes_cleaned[quizzes_cleaned['courseId'].isin(valid_courses)]

# Saving cleaned quizzes data
prep_quizzes_file_path = './data/prep_quizzes.csv'
quizzes_cleaned.to_csv(prep_quizzes_file_path, index=False)
print(f"Quizzes data cleaned and saved to {prep_quizzes_file_path} successfully.")

Quizzes data cleaned and saved to ./data/prep_quizzes.csv successfully.


## For questions

In [34]:
questions_raw = pd.read_csv(questions_raw_file)

# Cleaning Questions
questions_cleaned = questions_raw.drop_duplicates(subset=['quizId'])
questions_cleaned = questions_cleaned.dropna(subset=['quizId', 'questionText', 'answerA', 'answerB', 'answerC', 'answerD', 'correctAnswer'])

# Check if quizId exists in the quizzes dataset
quizzes_df = pd.read_csv('./data/prep_quizzes.csv')
valid_quizzes = quizzes_df['id'].unique()
questions_cleaned = questions_cleaned[questions_cleaned['quizId'].isin(valid_quizzes)]

# Saving cleaned questions data
prep_questions_file_path = './data/prep_questions.csv'
questions_cleaned.to_csv(prep_questions_file_path, index=False)
print(f"Questions data cleaned and saved to {prep_questions_file_path} successfully.")

Questions data cleaned and saved to ./data/prep_questions.csv successfully.


## For results data

In [35]:
results_raw = pd.read_csv(results_raw_file)

# Cleaning Results
results_cleaned = results_raw.drop_duplicates(subset=['userId', 'quizId'])
results_cleaned = results_cleaned.dropna(subset=['userId', 'quizId', 'score', 'totalScore'])

# Check if userId exists in the users dataset
users_df = pd.read_csv('./data/prep_users.csv')
valid_users = users_df['id'].unique()
results_cleaned = results_cleaned[results_cleaned['userId'].isin(valid_users)]

# Check if quizId exists in the quizzes dataset
quizzes_df = pd.read_csv('./data/prep_quizzes.csv')
valid_quizzes = quizzes_df['id'].unique()
results_cleaned = results_cleaned[results_cleaned['quizId'].isin(valid_quizzes)]

# Saving cleaned results data
prep_results_file_path = './data/prep_results.csv'
results_cleaned.to_csv(prep_results_file_path, index=False)
print(f"Results data cleaned and saved to {prep_results_file_path} successfully.")

Results data cleaned and saved to ./data/prep_results.csv successfully.


## For posts data

In [36]:
posts_raw = pd.read_csv(posts_raw_file)

# Cleaning Posts
posts_cleaned = posts_raw.drop_duplicates(subset=['userId', 'courseId'])
posts_cleaned = posts_cleaned.dropna(subset=['content', 'userId', 'courseId'])

# Optional: Check if userId exists in the users dataset
users_df = pd.read_csv('./data/prep_users.csv')
valid_users = users_df['id'].unique()
posts_cleaned = posts_cleaned[posts_cleaned['userId'].isin(valid_users)]

# Convert createdAt to datetime
posts_cleaned['createdAt'] = pd.to_datetime(posts_cleaned['createdAt'], errors='coerce')


# Optional: Check if courseId exists in the courses dataset
courses_df = pd.read_csv('./data/prep_courses.csv')
valid_courses = courses_df['id'].unique()
posts_cleaned = posts_cleaned[posts_cleaned['courseId'].isin(valid_courses)]

# Saving cleaned posts data
prep_posts_file_path = './data/prep_posts.csv'
posts_cleaned.to_csv(prep_posts_file_path, index=False)
print(f"Posts data cleaned and saved to {prep_posts_file_path} successfully.")


Posts data cleaned and saved to ./data/prep_posts.csv successfully.


## For feedbacks data

In [37]:
feedbacks_raw = pd.read_csv(feedbacks_raw_file)

# Cleaning Feedback
feedbacks_cleaned = feedbacks_raw.drop_duplicates(subset=['userId', 'courseId'])
feedbacks_cleaned = feedbacks_cleaned.dropna(subset=['content', 'rating', 'userId', 'courseId'])

# Convert createdAt to datetime
feedbacks_cleaned['createdAt'] = pd.to_datetime(feedbacks_cleaned['createdAt'], errors='coerce')

# Optional: Check if userId exists in the users dataset
users_df = pd.read_csv('./data/prep_users.csv')
valid_users = users_df['id'].unique()
feedbacks_cleaned = feedbacks_cleaned[feedbacks_cleaned['userId'].isin(valid_users)]

# Optional: Check if courseId exists in the courses dataset
courses_df = pd.read_csv('./data/prep_courses.csv')
valid_courses = courses_df['id'].unique()
feedbacks_cleaned = feedbacks_cleaned[feedbacks_cleaned['courseId'].isin(valid_courses)]

# Saving cleaned feedbacks data
prep_feedbacks_file_path = './data/prep_feedbacks.csv'
feedbacks_cleaned.to_csv(prep_feedbacks_file_path, index=False)
print(f"Feedback data cleaned and saved to {prep_feedbacks_file_path} successfully.")

Feedback data cleaned and saved to ./data/prep_feedbacks.csv successfully.


## For user session data

In [38]:
user_sessions_raw = pd.read_csv(user_sessions_raw_file)

# Cleaning User Sessions
user_sessions_cleaned = user_sessions_raw.drop_duplicates(subset=['userId'])
user_sessions_cleaned = user_sessions_cleaned.dropna(subset=['userId', 'duration'])

# Optional: Check if userId exists in the users dataset
users_df = pd.read_csv('./data/prep_users.csv')
valid_users = users_df['id'].unique()
user_sessions_cleaned = user_sessions_cleaned[user_sessions_cleaned['userId'].isin(valid_users)]

# Saving cleaned user sessions data
prep_user_sessions_file_path = './data/prep_user_sessions.csv'
user_sessions_cleaned.to_csv(prep_user_sessions_file_path, index=False)
print(f"User Sessions data cleaned and saved to {prep_user_sessions_file_path} successfully.")

User Sessions data cleaned and saved to ./data/prep_user_sessions.csv successfully.
